In [ ]:
# Removes large-scale background gradients from the input flc files

from astropy.convolution import Gaussian2DKernel
from astropy.io import fits
from astropy.stats import gaussian_fwhm_to_sigma, SigmaClip, sigma_clip
import glob
import numpy as np
from photutils import Background2D, detect_sources, detect_threshold, MedianBackground

################################# USER INPUTS #################################

# The files to remove the background gradient from
files = glob.glob('./test_new/*flc.fits')

# The box size to use when creating the 2D background image
box_size = (128, 128)

# Option to mask sources when finding the background gradient
mask_sources = True

###############################################################################

for f in files:
    basename = os.path.basename(f)
    print('Working on {}:'.format(basename))
    h = fits.open(f)
    for ext in [1,4]:
        print('\tWorking on extension {}:'.format(ext))
        data_orig = np.copy(h[ext].data)
        data = h[ext].data

        # Subtract off median
        clipped = sigma_clip(data, sigma=3, maxiters=5)
        data = data - np.nanmedian(clipped)

        # Find sources in the gradient-removed image
        if mask_sources:
            print('\tMaking source segmap...')
            s = SigmaClip(sigma=3.)
            bkg_estimator = MedianBackground()
            bkg = Background2D(data, box_size=box_size, filter_size=(10, 10), 
                               sigma_clip=s, bkg_estimator=bkg_estimator)
            skydark = bkg.background
            data_flat = data_orig - skydark
            threshold = detect_threshold(data_flat, nsigma=1.0)
            sigma = 3.0 * gaussian_fwhm_to_sigma
            kernel = Gaussian2DKernel(sigma, x_size=3, y_size=3)
            kernel.normalize()
            segm = detect_sources(data_flat, threshold, npixels=5, filter_kernel=kernel)
            segmap = segm.data
            fits.writeto(f.replace('_flc.fits', '_segmap_ext{}.fits'.format(ext)), 
                         segmap, overwrite=True)
        else:
            segmap = np.zeros(data.shape).astype(int)
        
        # Find the background gradient, incorporating the source mask
        print('\tFinding the background gradient...')
        s = SigmaClip(sigma=3.)
        bkg_estimator = MedianBackground()
        mask = (segmap > 0)
        bkg = Background2D(data, box_size=box_size, filter_size=(10, 10), 
                           sigma_clip=s, bkg_estimator=bkg_estimator, mask=mask)
        skydark = bkg.background
        fits.writeto(f.replace('_flc.fits', '_bkg_ext{}.fits'.format(ext)), 
                     skydark, overwrite=True)

        # Subtract the background gradient from the original image
        data_new = data_orig - skydark
        h[ext].data = data_new.astype('float32')

    h.writeto(f, overwrite=True)
    h.close()
    print('Finished removing background gradient from {}'.format(basename))
